## COVID-19 cases in the UK

In [24]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import geopandas as gpd
from datetime import datetime
import os

Get daily indicators from Public Health England.

This data is provided as an Excel file - stream the binary data directly from this URL.

In [25]:
WORKING_PATH = os.getcwd() + '/'
DATA_PATH = os.path.dirname(os.getcwd()) + '/data/'

In [26]:
url = 'https://www.arcgis.com/sharing/rest/content/items/bc8ee90225644ef7a6f4dd1b13ea1d67/data'
# read .xlsx file from server and write to local file
r = requests.get(url, stream=True)
with open(WORKING_PATH + '~data.xlsx', 'wb') as f:
    for chunk in r.iter_content(chunk_size=1024): 
        if chunk: # filter out keep-alive new chunks
            f.write(chunk)
# read table from local file
di = pd.read_excel(WORKING_PATH + '~data.xlsx')
di.dtypes

XLRDError: Unsupported format, or corrupt file: Expected BOF record; found b'\n\n\n\n\n\n\n\n'

In [ ]:
# Total number of cases in England
total_cases = di['EnglandCases'].item()
# Date when last updated
dt = di['DateVal'].item()
total_cases, dt

Get data on COVID-19 cases by area from Public Health England.

This data is provided in csv format and updated daily.

In [ ]:
# load cases by Upper Tier Local Authority and Unitary Authorities
url = "https://www.arcgis.com/sharing/rest/content/items/b684319181f94875a6879bbc833ca3a6/data"
r = requests.get(url)
filename = WORKING_PATH + '~cases-data.csv'
with open(filename, 'w') as f:
    f.write(r.text)
cases_df = pd.read_csv(filename)
cases_df.head()

In [ ]:
cases_df[cases_df["GSS_NM"]=="Cornwall and Isles of Scilly"]

In [ ]:
cases_df[cases_df["GSS_NM"]=="Bournemouth, Christchurch and Poole"]

Get population data from local csv file (downloaded from ONS to local filesystem as static)

In [ ]:
popn_df = pd.read_csv(DATA_PATH + 'UK population.csv')
popn_df.head()

In [ ]:
# convert population data to float (removing comma thousand separators)
popn_df["2020"] = popn_df["2020"].str.replace(",","").astype(float)
# filter only 'All ages'
popn_df = popn_df[popn_df['AGE GROUP']=='All ages']
# get total population for England for later
england_popn = popn_df[popn_df['AREA']=='England']['2020']

Note Cornwall (E06000052), Isles of Scilly (E06000053), Bournemouth (E06000028), Christchurch (E07000048) & Poole (E06000029) are all separate entries in the population table - they are combined as "Cornwall and Isles of Scilly (E06000052)" and "Bournemouth, Christchurch & Poole (E06000058)" in the cases data.

In [ ]:
# filter out unwanted columns, and sum Cornwall & Isles of Scilly (as E06000052), and Bournemouth, Christchurch & Poole (as E06000058)
popn_df = popn_df.replace({"CODE": {
        "E06000053": "E06000052",
        "E06000028": "E06000058",
        "E07000048": "E06000058",
        "E06000029": "E06000058"
    }}).filter(items=['CODE', '2020']).groupby('CODE').sum()
popn_df.head()

Import shapefile for English counties and unitary authorities (includes boundaries and centroids).

In [ ]:
path = DATA_PATH + "maps/Counties_and_Unitary_Authorities_December_2016_Generalised_Clipped_Boundaries_in_England_and_Wales.shp"
gdf = gpd.read_file(path)
# check data type so we can see that this is not a normal dataframe, but a GEOdataframe - see the 'geometry' column contains shapes
gdf.dtypes

In [ ]:
# To-do: combine polygons for Bournemouth, Christchurch & Poole, and replace code with code for this combined area in cases data (E06000058)
gdf[gdf['ctyua16nm']=='Bournemouth']

In [ ]:
# include only English areas
gdf = gdf[gdf['ctyua16cd'].str.contains("E")]

In [ ]:
# join population data with shapes, matching on index columns
gdf = gdf.set_index('ctyua16cd').merge(popn_df, left_index=True, right_index=True, how='left')
gdf = gdf.replace({'ctyua16nm': {'Cornwall': 'Cornwall and Isles of Scilly'}})
gdf.head()

In [ ]:
# join cases data with shapes
cases_df.rename(columns={"GSS_NM": "ctyua16nm"}, inplace=True)
# use left join to preserve rows present in gdf but missing from cases_df (default is 'inner')
gdf = gdf.merge(cases_df, on="ctyua16nm", how="left")
# convert TotalCases column to float
gdf["TotalCases"] = gdf["TotalCases"].str.replace(",","").astype(float)
gdf.head()

Calculate cases per 1000 population.

In [ ]:
gdf.rename(columns={"2020": "Population"}, inplace=True)
gdf['density'] = gdf['TotalCases'] / gdf['Population']

Normalise using the number of cases per 1,000 population in England. 

In [ ]:
total_utua_cases = gdf['TotalCases'].sum()
mean_density = total_utua_cases / england_popn

In [ ]:
hot_spots = gdf.sort_values(by=['density'], ascending=False)['ctyua16nm'].head(3).to_list()

In [ ]:
from mpl_toolkits.axes_grid1 import make_axes_locatable

fig, ax = plt.subplots(1, figsize=(10, 15))
# set up axes to enable positioning of the color bar.
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=1)

#set fixed range for colors
vmin, vmax = 0, 0.1

ax.axis('off')
ax.set_aspect('equal')
ax.set_title('Distribution of confirmed cases of COVID-19 in England', fontdict={'fontsize': '16', 'fontweight' : '3'})
ax.annotate('Source: Public Health England, {}'.format(dt.strftime("%d %B %Y")),
            xy=(0.1, 0.08),
            xycoords='figure fraction',
            horizontalalignment='left',
            verticalalignment='top',
            fontsize=12)
ax.annotate('Total number of cases: {}\nHot-spots: {}' \
            .format(total_cases, ", ".join(hot_spots)),
            xy=(0.1, 0.15), 
            xycoords='figure fraction', 
            horizontalalignment='left',
            verticalalignment='top',
            fontsize=12)

# ensure geometry is taken from the shapes for the base plot
gdf = gdf.set_geometry('geometry')
# base plot is a chloropleth using values from the 'density' column
base = gdf.plot(ax=ax, cax=cax, column='density', legend=True, linewidth=0.5, edgecolor="0.8",
                legend_kwds={'label': "Cases per 1,000 population"}, cmap="YlOrRd", vmin=0, vmax=1, 
                missing_kwds={"color": "lightgrey", "label": "No data"})
# calculate x,y coordinates of centroids of areas (for scatter plot)
gdf['centroids'] = gdf.centroid
# change the geometry column to 'centroids'
gdf = gdf.set_geometry('centroids')
# second layer is a scatter plot taking position from 'centroids' and marker size from 'TotalCases'
plot = gdf.plot(ax=base, color="blue", alpha=0.5, markersize=gdf["TotalCases"])

# produce legend for size of markers
for number in [1, 50, 100]:
    plt.scatter([], [], c='blue', alpha=0.5, s=number, label=str(number))
plt.legend(scatterpoints=1, frameon=True, labelspacing=1, title='Number of cases')

#plt.show()

In [ ]:
fig.savefig(WORKING_PATH + 'charts/covid-19-england-{}.png'.format(dt.strftime("%Y-%B-%d")), dpi=300)

In [ ]:
gdf.sort_values(by=['TotalCases'], ascending=False).filter(['TotalCases', 'ctyua16nm']).head()

In [ ]:
# create simple html page to view daily charts

from jinja2 import Template
from os import listdir, path

# list files in target directory, excluding hidden files.
files = sorted(['/'.join(['charts', f]) for f in listdir('charts') if f[0] != '.'])

# render template
with open(WORKING_PATH + "template.html", "r") as f:
    t = Template(f.read())

with open(WORKING_PATH + "index.html", "w") as f:
    f.write(t.render(files=files))

In [ ]:
# update README.md with link to latest chart
lines=[]
with open(WORKING_PATH + 'README.md', 'r') as file:
    for line in file:
        lines.append(line)

# Find and replace the target string
newdata = ''
flag = False
for line in lines:
    if flag:
        line = "![{}](charts/covid-19-england-{}.png)\n" \
            .format(dt.strftime("%d %B %Y"), dt.strftime("%Y-%B-%d"))
        flag = False
    newdata += line
    if line=="## Results\n":
        flag = True

# Write the file out again
with open(WORKING_PATH + 'README.md', 'w') as file:
    file.write(newdata)